In [19]:
# 0605.py
import cv2
import numpy as np

#1
src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
blur= cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0.0)
cv2.imshow('src',  src)
cv2.imshow('blur', blur)

#2
lap  = cv2.Laplacian(src, cv2.CV_32F)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(lap)
print('lap:', minVal, maxVal, minLoc, maxLoc)
dst = cv2.convertScaleAbs(lap)
dst = cv2.normalize(dst, None, 0, 255, cv2.NORM_MINMAX)
cv2.imshow('lap',  lap)
cv2.imshow('dst',  dst)

#3
lap2 = cv2.Laplacian(blur, cv2.CV_32F)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(lap2)
print('lap2:', minVal, maxVal, minLoc, maxLoc)
dst2 = cv2.convertScaleAbs(lap2)
dst2 = cv2.normalize(dst2, None, 0, 255, cv2.NORM_MINMAX)

cv2.imshow('lap2',  lap2)
cv2.imshow('dst2',  dst2)

cv2.waitKey()    
cv2.destroyAllWindows()


lap: -239.0 189.0 (404, 181) (221, 287)
lap2: -35.0 30.0 (180, 435) (367, 449)


In [57]:
# 0606.py1
import numpy as np
import cv2
#1
src  = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
#src  = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)
blur = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0.0)
lap = cv2.Laplacian(blur, cv2.CV_32F,3)

##ret, edge = cv2.threshold(np.abs(lap), 10, 255, cv2.THRESH_BINARY)
##edge = edge.astype(np.uint8)
##cv2.imshow('edge',  edge)

#2
def SGN(x):
    if x >= 0:
        sign = 1
    else:
        sign = -1
    return sign

def zeroCrossing(lap):
    width, height = lap.shape
    Z = np.zeros(lap.shape, dtype=np.uint8)    
    for y in range(1, height-1):
        for x in range(1,width-1):
            neighbors=[lap[y-1,x], lap[y+1,x], lap[y,x-1], lap[y,x+1],
                       lap[y-1,x-1], lap[y-1,x+1], lap[y+1,x-1], lap[y+1,x+1]]                       
            mValue= min(neighbors)
            if SGN(lap[y,x]) != SGN(mValue):
                Z[y, x] = 255
    return Z
edgeZ = zeroCrossing(lap)
cv2.imshow('Zero Crossing',  edgeZ)
cv2.waitKey()    
cv2.destroyAllWindows()


In [49]:
# 0607.py
import cv2
import numpy as np

##src  = cv2.imread('./data/rect.jpg', cv2.IMREAD_GRAYSCALE)
src  = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

#1
kx, ky = cv2.getDerivKernels(1, 0, ksize=3)
sobelX = ky.dot(kx.T)
print('kx=', kx)
print('ky=', ky)
print('sobelX=', sobelX)
##gx = cv2.filter2D(src, cv2.CV_32F,sobelX)
gx = cv2.sepFilter2D(src, cv2.CV_32F, kx, ky)

#2
kx, ky = cv2.getDerivKernels(0, 1, ksize=3)
sobelY = ky.dot(kx.T)
print('kx=', kx)
print('ky=', ky)
print('sobelY=', sobelY)
##gy = cv2.filter2D(src, cv2.CV_32F, sobelY)
gy = cv2.sepFilter2D(src, cv2.CV_32F, kx, ky)

#3
mag   = cv2.magnitude(gx, gy)
ret, edge = cv2.threshold(mag, 100, 255, cv2.THRESH_BINARY)
cv2.imshow('gx',gx)
cv2.imshow('gy',gy)
cv2.imshow('edge',  edge)
cv2.waitKey()    
cv2.destroyAllWindows()


kx= [[-1.]
 [ 0.]
 [ 1.]]
ky= [[1.]
 [2.]
 [1.]]
sobelX= [[-1.  0.  1.]
 [-2.  0.  2.]
 [-1.  0.  1.]]
kx= [[1.]
 [2.]
 [1.]]
ky= [[-1.]
 [ 0.]
 [ 1.]]
sobelY= [[-1. -2. -1.]
 [ 0.  0.  0.]
 [ 1.  2.  1.]]


In [56]:
# 0608.py
import cv2
import numpy as np

src  = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

#1
def logFilter(ksize=7):
    k2 = ksize//2
    sigma = 0.3*(k2-1) + 0.8
    print('sigma=', sigma)
    LoG = np.zeros((ksize, ksize), dtype=np.float32)
    
    for y in range(-k2, k2+1):
        for x in range(-k2, k2+1):
            g = -(x*x+y*y)/(2.0*sigma**2.0)
            LoG[y+k2, x+k2] = -(1.0+g)*np.exp(g)/(np.pi*sigma**4.0)
    return LoG

#2
kernel = logFilter() #7, 15, 31, 51

LoG = cv2.filter2D(src, cv2.CV_32F, kernel)
cv2.imshow('LoG',  LoG)

blur = cv2.GaussianBlur(src, ksize=(7, 7), sigmaX=0.0)
lap = cv2.Laplacian(blur, cv2.CV_32F,3)

cv2.imshow('LoG2',lap)
#3
def zeroCrossing2(lap, thresh=0.01):
    width, height = lap.shape
    Z = np.zeros(lap.shape, dtype=np.uint8)    
    for y in range(1, height-1):
        for x in range(1,width-1):
            neighbors=[lap[y-1,x],   lap[y+1,x],   lap[y,x-1],   lap[y,x+1],
                       lap[y-1,x-1], lap[y-1,x+1], lap[y+1,x-1], lap[y+1,x+1]]
            pos = 0
            neg = 0
            for value in neighbors:
                if value > thresh:
                    pos += 1
                if value < -thresh:  # value < thresh
                    neg += 1
            if pos > 0 and neg > 0:
                Z[y, x] = 255                        
    return Z
edgeZ = zeroCrossing2(LoG)
a_edge=zeroCrossing(lap)
cv2.imshow('Zero2',a_edge)
cv2.imshow('Zero',  edgeZ)
cv2.waitKey()    
cv2.destroyAllWindows()


sigma= 1.4


In [68]:
# 0609.py
import cv2
import numpy as np

src   = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)
kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))
erode = cv2.erode(src,kernel,iterations = 5)
dilate = cv2.dilate(src,kernel,iterations = 5)
erode2= cv2.erode(dilate,kernel,iterations = 7)
##dilate2= cv2.dilate(erode2,kernel,iterations = 2)

cv2.imshow('src',    src)
cv2.imshow('erode',  erode)
cv2.imshow('dilate', dilate)
cv2.imshow('erode2', erode2)
##cv2.imshow('dilate2',    dilate2)
cv2.waitKey()
cv2.destroyAllWindows()


In [72]:
# 0610.py
import cv2
import numpy as np

src   = cv2.imread('./data/morphology.jpg', cv2.IMREAD_GRAYSCALE)
kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(3,3))
closing = cv2.morphologyEx(src,cv2.MORPH_CLOSE,kernel,iterations = 5)
opening = cv2.morphologyEx(closing,cv2.MORPH_OPEN,kernel,iterations = 5)
gradient = cv2.morphologyEx(opening,cv2.MORPH_GRADIENT,kernel)
#gradient=cv2.morphologyEx(opening,cv2.MORPH_GRADIENT,kernel, iterations=5)

tophat = cv2.morphologyEx(src,cv2.MORPH_TOPHAT,kernel,iterations = 5)
balckhat = cv2.morphologyEx(src,cv2.MORPH_BLACKHAT,kernel,iterations = 5)

cv2.imshow('opening',  opening)
cv2.imshow('closing',  closing)
cv2.imshow('gradient', gradient)
cv2.imshow('tophat',   tophat)
cv2.imshow('balckhat', balckhat)
cv2.waitKey()
cv2.destroyAllWindows()


In [102]:
# 0611.py
"""
ref: https://gist.github.com/jsheedy/3913ab49d344fac4d02bcc887ba4277d
ref: http://felix.abecassis.me/2011/09/opencv-morphological-skeleton/
"""
import cv2
import numpy as np

#1
src   = cv2.imread('./data/T.jpg', cv2.IMREAD_GRAYSCALE)
##src   = cv2.imread('alphabet.bmp', cv2.IMREAD_GRAYSCALE)
##src = cv2.bitwise_not(src)

ret, A = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY)
skel_dst = np.zeros(src.shape, np.uint8)

#2
shape1=cv2.MORPH_CROSS
shape2=cv2.MORPH_RECT
B= cv2.getStructuringElement(shape=shape1, ksize=(3,3))
##B[2,2]=4
done = True
while done:   
    erode  = cv2.erode(A, B)
##    opening = cv2.dilate(erode,B)
    opening = cv2.morphologyEx(erode, cv2.MORPH_OPEN, B)
    tmp    = cv2.subtract(erode, opening) # cv2.absdiff(erode, opening)
    skel_dst = cv2.bitwise_or(skel_dst, tmp)
    ##cv2.imshow('opening',opening)
    ##cv2.imshow('tmp',tmp)
    cv2.imshow('skel',skel_dst)
    cv2.waitKey(100)
    A = erode.copy()
    done = cv2.countNonZero(A) != 0
    
##    cv2.imshow('opening',  opening)
##    cv2.imshow('tmp',  tmp)    
##    cv2.imshow('skel_dst',  skel_dst)
##    cv2.waitKey()

cv2.imshow('src',  src)    
cv2.imshow('skel_dst',  skel_dst)
cv2.waitKey()
cv2.destroyAllWindows()


In [109]:
# 0612.py
import cv2
import numpy as np

src   = cv2.imread('./data/alphabet.bmp', cv2.IMREAD_GRAYSCALE)
tmp_A   = cv2.imread('./data/A.bmp', cv2.IMREAD_GRAYSCALE)
tmp_S   = cv2.imread('./data/S.bmp', cv2.IMREAD_GRAYSCALE)
tmp_b   = cv2.imread('./data/b.bmp', cv2.IMREAD_GRAYSCALE)
dst  = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)  # 출력 표시 영상

#1
R1 = cv2.matchTemplate(src, tmp_A, cv2.TM_SQDIFF_NORMED)
minVal, _, minLoc, _ = cv2.minMaxLoc(R1)
print('TM_SQDIFF_NORMED:', minVal, minLoc)

w, h = tmp_A.shape[:2]
cv2.rectangle(dst, minLoc, (minLoc[0]+h, minLoc[1]+w), (255, 0, 0), 2)

#2
R2 = cv2.matchTemplate(src, tmp_S, cv2.TM_CCORR_NORMED)
_, maxVal, _, maxLoc = cv2.minMaxLoc(R2)
print('TM_CCORR_NORMED:', maxVal, maxLoc)
w, h = tmp_S.shape[:2]
cv2.rectangle(dst, maxLoc, (maxLoc[0]+h, maxLoc[1]+w), (0, 255, 0), 2)

#3
R3 = cv2.matchTemplate(src, tmp_b, cv2.TM_CCOEFF_NORMED)
_, maxVal, _, maxLoc = cv2.minMaxLoc(R3)
print('TM_CCOEFF_NORMED:', maxVal, maxLoc)
w, h = tmp_b.shape[:2]
cv2.rectangle(dst, maxLoc, (maxLoc[0]+h, maxLoc[1]+w), (0, 0, 255), 2)

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()


TM_SQDIFF_NORMED: 0.0 (18, 20)
TM_CCORR_NORMED: 1.000000238418579 (280, 146)
TM_CCOEFF_NORMED: 0.9999994039535522 (92, 280)


In [110]:
# 0701.py
import cv2
import numpy as np

src = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

edges1 = cv2.Canny(src, 50, 100)
edges2 = cv2.Canny(src, 50, 200)
 
cv2.imshow('edges1',  edges1)
cv2.imshow('edges2',  edges2)
cv2.waitKey()
cv2.destroyAllWindows()


In [117]:
# 0702.py
import cv2
import numpy as np

src = cv2.imread('./data/rect.jpg')
gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 100)
lines = cv2.HoughLines(edges, rho=1, theta=np.pi/180.0, threshold=100)
print('linebs.shape=', lines.shape)

for line in lines:
    rho, theta   = line[0]
    c = np.cos(theta)
    s = np.sin(theta)
    x0 = c*rho
    y0 = s*rho
    x1 = int(x0 + 1000*(-s))
    y1 = int(y0 + 1000*(c))
    x2 = int(x0 - 1000*(-s))
    y2 = int(y0 - 1000*(c))
    cv2.line(src, (x1,y1), (x2,y2), (0,0,255), 2)
    
cv2.imshow('edges',  edges)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()


linebs.shape= (4, 1, 2)


In [124]:
# 0704.py
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/circles.jpg')
gray1 = cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
circles1 = cv2.HoughCircles(gray1, method = cv2.HOUGH_GRADIENT,
            dp=1, minDist=50, param2=15)

print('circles1.shape=', circles1.shape)
for circle in circles1[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src1, (cx, cy), r, (0,0,255), 2)
cv2.imshow('src1',  src1)

#2
src2 = cv2.imread('./data/circles2.jpg')
gray2 = cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)
circles2 = cv2.HoughCircles(gray2, method = cv2.HOUGH_GRADIENT,
          dp=1, minDist=50, param2=15, minRadius=30, maxRadius=100)

print('circles2.shape=', circles2.shape)
for circle in circles2[0,:]:    
    cx, cy, r  = circle
    cv2.circle(src2, (cx, cy), r, (0,0,255), 2) 
cv2.imshow('src2',  src2)
cv2.waitKey()
cv2.destroyAllWindows()


circles1.shape= (1, 3, 3)
circles2.shape= (1, 6, 3)


In [131]:
# 0705.py
import cv2
import numpy as np

#1
src1 = cv2.imread('./data/hand.jpg')
hsv1 = cv2.cvtColor(src1, cv2.COLOR_BGR2HSV)
lowerb1 = (0, 40, 0)
upperb1 = (20, 180, 255)
dst1 = cv2.inRange(hsv1, lowerb1, upperb1)

#2
src2 = cv2.imread('./data/flower.jpg')
hsv2 = cv2.cvtColor(src2,cv2.COLOR_BGR2HSV)
lowerb2 = (150, 100, 100)
upperb2 = (180, 255, 255)
dst2 = cv2.inRange(hsv2, lowerb2, upperb2)

#3
cv2.imshow('src1',  src1)
cv2.imshow('dst1',  dst1)
cv2.imshow('src2',  src2)
cv2.imshow('dst2',  dst2)
cv2.waitKey()
cv2.destroyAllWindows()


In [143]:
# 0706.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512,3), dtype=np.uint8)
cv2.rectangle(src, (50, 100), (450, 400), (255, 255, 255), -1)
cv2.rectangle(src, (100, 150), (400, 350), (0, 0, 0), -1)
cv2.rectangle(src, (200, 200), (300, 300), (255, 255, 255), -1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#2
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
##method =cv2.CHAIN_APPROX_NONE
image, contours, hierarchy = cv2.findContours(gray, mode, method)
print('type(contours)=', type(contours))
print('type(contours[0])=', type(contours[0]))
print('len(contours)=', len(contours))
print('contours[0].shape=', contours[0].shape)
print('contours[0]=', contours[0])

#3
cv2.drawContours(src, contours, -1, (255,0,0), 3) # 모든 윤곽선

#4
for pt in contours[0][:]: # 윤곽선 좌표
    cv2.circle(src, (pt[0][0], pt[0][1]), 5, (0,0,255), -1)
cv2.imshow('gray',gray)
cv2.imshow('src',  src)
cv2.waitKey()
cv2.destroyAllWindows()
\

type(contours)= <class 'list'>
type(contours[0])= <class 'numpy.ndarray'>
len(contours)= 1
contours[0].shape= (4, 1, 2)
contours[0]= [[[ 50 100]]

 [[ 50 400]]

 [[450 400]]

 [[450 100]]]


In [146]:
# 0708.py
import cv2
import numpy as np

#1
src = np.full((512,512,3), (255, 255, 255), dtype= np.uint8)
cv2.rectangle(src, (50, 50), (200, 200), (0, 0, 255), 2)
cv2.circle(src, (300, 300), 100, (0,0,255), 2)

#2
dst = src.copy()
cv2.floodFill(dst, mask=None, seedPoint=(100,100), newVal=(255,0,0))

#3
retval, dst2, mask, rect=cv2.floodFill(dst, mask=None,
                          seedPoint=(300,300), newVal=(0,255,0))
print('rect=', rect)
x, y, width, height = rect
cv2.rectangle(dst2, (x,y), (x+width, y+height), (255, 0, 0), 2)

cv2.imshow('src',  src)
cv2.imshow('dst',  dst)
cv2.imshow('dst2',dst2)
cv2.waitKey()
cv2.destroyAllWindows()


rect= (202, 202, 197, 197)


In [147]:
# 0709.py
import cv2
import numpy as np

#1
src = np.zeros(shape=(512,512), dtype=np.uint8)
cv2.rectangle(src, (50, 200), (450, 300), (255, 255, 255), -1)

#2
dist  = cv2.distanceTransform(src, distanceType=cv2.DIST_L1, maskSize=3)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(dist)
print('src:', minVal, maxVal, minLoc, maxLoc)

dst = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
ret, dst2 = cv2.threshold(dist, maxVal-1, 255, cv2.THRESH_BINARY)

#3 
gx = cv2.Sobel(dist, cv2.CV_32F, 1, 0, ksize = 3)
gy = cv2.Sobel(dist, cv2.CV_32F, 0, 1, ksize = 3)
mag   = cv2.magnitude(gx, gy)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(mag)
print('src:', minVal, maxVal, minLoc, maxLoc)
ret, dst3 = cv2.threshold(mag, maxVal-2, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('src',  src)
cv2.imshow('dst',  dst)
cv2.imshow('dst2',  dst2)
cv2.imshow('dst3',  dst3)
cv2.waitKey()
cv2.destroyAllWindows()


src: 0.0 51.0 (0, 0) (100, 250)
src: 0.0 8.0 (0, 0) (52, 200)
